In [ ]:
import requests
import json
import pandas as pd

# Informations du client
client_id = 'm7eccvja28j882ndzn4h46fuh'
client_secret = 'nu228RpY9M'

# Obtenir le jeton d'accès
url_token = "https://api.lufthansa.com/v1/oauth/token"
headers_token = {
    'Content-Type': 'application/x-www-form-urlencoded'
}
data_token = {
    'client_id': client_id,
    'client_secret': client_secret,
    'grant_type': 'client_credentials'
}

response_token = requests.post(url_token, headers=headers_token, data=data_token)
if response_token.status_code == 200:
    token_info = response_token.json()
    access_token = token_info['access_token']
    print(f"Access Token: {access_token}")
else:
    raise Exception(f"Failed to get access token: {response_token.status_code} {response_token.text}")

# Headers communs pour les requêtes
headers_common = {
    'Authorization': f'Bearer {access_token}',
    'Accept': 'application/json'
}

In [ ]:
all_arrival_actual_time = []
all_arrival_scheduled_time = []
all_arrival_airport_code = []
all_arrival_ts_code = []

all_departure_actual_time = []
all_departure_scheduled_time = []
all_departure_airport_code = []
all_departure_ts_code = []

all_aircraft_code = []
all_fs_code = []
all_service_type = []

fromDateTime = '2024-06-10T00:00'

airports_code = pd.read_csv("airports.csv")

for airport_code in list(airports_code["AirportCode"]) :
    
    url_departures = F"https://api.lufthansa.com/v1/operations/flightstatus/departures{airport_code}/{fromDateTime}"
    response_departures = requests.get(url_departures, headers=headers_common)
    departures_info = response_departures.json()
    departures_info = json.dumps(departures_info, indent=4, sort_keys=True)
    print("Departures Information:")
    print(departures_info)


    limit = 100
    offset = 0

    while True:
        params = {
            "limit": limit,
            "offset": offset,
            "lang": "en"
        }

        response_departures = requests.get(url_departures, headers=headers_common)
        data = response_departures.json()

        response_departures = data.get('FlightStatusResource', {}).get('Flights', {}).get('Flight', {})

        for response in response_departures :

            all_arrival_actual_time.append(response.get('Arrival',{}).get('ActualTimeUTC',{}).get('DateTime'))
            all_arrival_scheduled_time.append(response.get('Arrival',{}).get('ScheduledTimeUTC',{}).get('DateTime'))
            all_arrival_airport_code.append(response.get('Arrival',{}).get('AirportCode'))
            all_arrival_ts_code.append(response.get('Arrival',{}).get('TimeStatus',{}).get('Code'))

            all_departure_actual_time.append(response.get('Departure',{}).get('ActualTimeUTC',{}).get('DateTime'))
            all_departure_scheduled_time.append(response.get('Departure',{}).get('ScheduledTimeUTC',{}).get('DateTime'))
            all_departure_airport_code.append(response.get('Departure',{}).get('AirportCode'))
            all_departure_ts_code.append(response.get('Departure',{}).get('TimeStatus',{}).get('Code'))

            all_aircraft_code.append(response.get('Equipment',{}).get('AircraftCode'))
            all_fs_code.append(response.get('FlightStatus',{}).get('Code'))
            all_service_type.append(response.get('ServiceType'))

        offset += limit

        if len(response) < limit:
            break

In [ ]:
df = {
    'ArrivalActualTimeUTC' : all_arrival_actual_time,
    'ArrivalScheduledTimeUTC' : all_arrival_scheduled_time,
    'ArrivalAirportCode' : all_arrival_airport_code,
    'ArrivalTimeStatus' : all_arrival_ts_code,
    'DepartureActualTimeUTC' : all_departure_actual_time,
    'DepartureScheduledTimeUTC' : all_departure_scheduled_time,
    'DepartureAirportCode' : all_departure_airport_code,
    'DepartureTimeStatus' : all_departure_ts_code
}

data = pd.DataFrame(df)
data